In [1]:
import sys
import os
import json
import pandas as pd
import numpy as np

base_dir = os.path.abspath(os.path.join(os.getcwd(), "../../../"))
sys.path.append(os.path.join(base_dir, "Data_Augmentation"))

from llm import llm_response

print(os.getcwd())


/Users/tkang/Documents/research/nlp_followupqg/Human_Evaluation/Analysis/mode_3


In [4]:
full_df = pd.read_json("../../../Auto_Evaluation/full_clustered.json")
org_df = pd.read_json("../../../Auto_Evaluation/org_clustered.json")
gpt_df = pd.read_json("../../../Auto_Evaluation/gpt_clustered.json")

assert (len(full_df) == 501) and (len(full_df) == len(org_df)) and (len(full_df) == len(gpt_df))

In [5]:
with open("../../../Data_Augmentation/test_complete_answers.json", "r") as f:
# Data_Augmentation/test.json 
    complete_answer_data = pd.DataFrame(json.load(f))

assert (len(full_df) == len(complete_answer_data))

print(complete_answer_data.loc[0])

question                                        ELI5 Do animals tan?
answer             Animals can get sunburned like we do, pigs for...
complete_answer    Yes, some animals can tan. Similar to humans, ...
Name: 0, dtype: object


In [7]:
num_data = len(full_df)

# checking that the data are lined up properly
for i in range(num_data):
    assert (full_df.loc[i, 'question'] == org_df.loc[i, 'question']) and (full_df.loc[i, 'question'] == gpt_df.loc[i, 'question']) and (full_df.loc[i, 'question'] == complete_answer_data.loc[i, 'question'])

    assert (full_df.loc[i, 'answer'] == org_df.loc[i, 'answer']) and (full_df.loc[i, 'answer'] == gpt_df.loc[i, 'answer']) and (full_df.loc[i, 'answer'] == complete_answer_data.loc[i, 'answer'])

In [ ]:
# # Uncomment the code block below to run the api call again (it takes ~3 Hours to run the entire dataset)
# Call GPT API for each model's (Full, Org, GPT) OA/CA/FUQs Pairs
index_raises_error = []
num_data = len(full_df)

full_df_answerability_all = []
org_df_answerability_all = []
gpt_df_answerability_all = []

full_df_with_answerability = full_df.copy()
org_df_with_answerability = org_df.copy()
gpt_df_with_answerability = gpt_df.copy()

ERROR_MESSAGE = 'LLM failed to generate a response'

def get_llm_response(i, df, ca_df):
    OA = df.loc[i, 'answer']
    CA = ca_df.loc[i, "complete_answer"]
    FUQ = df.loc[i, "generated_follow_up"]

    question = f"Please tell me whether each of the questions in 'Follow Up Questions' is answerable by either/both the 'Original Answer' and the 'Complete Answer'. Original Answer: {OA}, Complete Answer: {CA}, Follow Up Questions: {FUQ}"
    prompt = """
        You must strictly follow these instructions:

        - Given an "Original Answer" and a "Complete Answer," evaluate a list of "Follow-Up Questions."
        - For each Follow-Up Question, determine whether it can be answered by:
            - The "Original Answer" only → Return ["Original Answer"]
            - The "Complete Answer" only → Return ["Complete Answer"]
            - Both answers → Return ["Original Answer", "Complete Answer"]
            - Neither answer → Return []

        ### **Criteria for Answerability (Assume Answerability Unless Completely Unrelated):**
        A Follow-Up Question (FUQ) is considered **answerable** if it contains **any relevant information** that connects to the answer in some way.  
        The answer does **not** need to be explicit, as long as it provides **some useful context** that **relates to** or **touches on** the FUQ's topic.

        1. **Direct Coverage** ✅  
        - The answer explicitly provides the required information.  
        - No inference is needed.  
        - Example:  
            - FUQ: "What is blackbody radiation?"  
            - OA: "Blackbody radiation occurs when hot objects emit their own light."  
            - ✅ Answerable by OA.  

        2. **Loose Inferability (Minimal Context Needed) ✅**  
        - The answer **does not directly answer** the FUQ but provides **some level of relevant context**, even if vague.  
        - If **some background knowledge** allows for a connection, it should be marked as answerable.  
        - Example:  
            - FUQ: "How do blackbody radiation and white light differ in intensity?"  
            - OA: "Sufficiently hot objects produce their own light, a process called blackbody radiation."  
            - ✅ Answerable by OA (since blackbody radiation involves emitted light, a connection to intensity can be made).  

        3. **Any Shared Concept Counts** ✅  
        - If the FUQ **mentions a topic** that appears in the OA or CA, it should be considered answerable.  
        - Even if the answer does **not fully explain** the FUQ, **any overlap** makes it answerable.  
        - Example:  
            - FUQ: "What are some examples of high-energy emitting materials?"  
            - OA: "Torches, incandescent bulbs, and the sun emit light through blackbody radiation."  
            - ✅ Answerable by OA (mention of blackbody radiation implies high-energy emission).  

        4. **Very Flexible Scope** ✅  
        - The FUQ **should be marked as answerable unless it is completely unrelated**.  
        - If **even a minor part** of the FUQ is **somewhat addressed**, it should be considered answerable.  
        - Example:  
            - FUQ: "How does thermal decomposition contribute to the emission of white light?"  
            - OA: "Sufficiently hot objects produce their own light."  
            - ✅ Answerable by OA (mention of heat and light emission allows for an inference).  

        5. **Extreme Leniency for Partial Answers** ✅  
        - A FUQ **does not have to be fully answered** to be considered answerable.  
        - Even if the answer **only hints at the topic** or **briefly addresses it**, it should be marked as answerable.  
        - Example:  
            - FUQ: "How does the wavelength of light affect its energy?"  
            - OA: "Sufficiently hot objects emit their own light."  
            - ✅ Answerable by OA (even though the full explanation isn't there, it touches on emitted light). 

        ### **STRICT OUTPUT FORMAT RULES (DO NOT VIOLATE)**
        1. **The output must be a single-line string list.**  
        - Example output:  
            "[[], ["Complete Answer"], [], ["Original Answer", "Complete Answer"], ["Original Answer"]]"  
        - 🚫 Do **not** return separate lines.  
        - 🚫 Do **not** return explanations, summaries, or any extra text.  

        2. **Every Follow-Up Question must have exactly one corresponding entry.**  
        - The length of the output list **must match** the number of FUQs provided.  

        3. **Do NOT include escape characters ('\') or newline characters ('\n').**  
        - The output should be clean JSON without unnecessary formatting issues.  

        4. **The response must be a SINGLE list with elements separated by comma (',') wrapping all results.**  
        - 🚫 Incorrect Example:  
            "["Original Answer", "Complete Answer"], ["Original Answer", "Complete Answer"], ["Complete Answer"], []"
        - ✅ Correct Example:  
            "[["Original Answer", "Complete Answer"], ["Original Answer", "Complete Answer"], ["Complete Answer"], []]"

        5. **Do NOT add any extra text.**  
        - **ONLY return the structured list** and nothing else.  

        6. **POST-PROCESSING:**
        - **After generating the response, apply this replacement rule to ensure clean formatting:**  
            response = response.replace("“", '"').replace("”", '"').replace("'", '"').replace("\n", "").replace("\\", "")
        - This ensures the output is **valid JSON format** without incorrect quotation marks.

        ### **Input Data:**
        Original Answer: {OA}  
        Complete Answer: {CA}  
        Follow-Up Questions: {FUQ}  

        Now, analyze the given Follow-Up Questions and return ONLY the correctly structured list and nothing more:
    """

    return llm_response(prompt, question)

def save_response_to_df(i, df, ca_df, response):
    df.loc[i, "complete_answer"] = ca_df.loc[i, "complete_answer"]
    df.loc[i, "generated_follow_up_answerability"] = response

for i in range(num_data):
# for i in range(5):
    # Full Dataset
    full_response = get_llm_response(i, full_df_with_answerability, complete_answer_data)
    org_response = get_llm_response(i, org_df_with_answerability, complete_answer_data)
    gpt_response = get_llm_response(i, gpt_df_with_answerability, complete_answer_data)

    if full_response == ERROR_MESSAGE or org_response == ERROR_MESSAGE or gpt_response == ERROR_MESSAGE:
        index_raises_error.append(i)

    # store llm output data (list of list)
    full_df_answerability_all.append(full_response)
    org_df_answerability_all.append(org_response)
    gpt_df_answerability_all.append(gpt_response)

    save_response_to_df(i, full_df_with_answerability, complete_answer_data, full_response)
    save_response_to_df(i, org_df_with_answerability, complete_answer_data, org_response)
    save_response_to_df(i, gpt_df_with_answerability, complete_answer_data, gpt_response)

Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': True, 'severity': 'medium'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param

In [57]:
full_df_with_answerability

,id,question,answer,follow-up,relation,generated_follow_up,complete_answer,generated_follow_up_answerability
0,3000,ELI5 Do animals tan?,"Animals can get sunburned like we do, pigs for...",But can they tan? Does their body create color...,Related,[What are the primary sources of heat for anim...,"Yes, some animals can tan. Similar to humans, ...","[[], [""Complete Answer""], [""Complete Answer""],..."
1,3001,ELI5 What is GERD?,Gerd (GastroEsophageal Reflux Disease) is a pr...,I c this was the answer I was looking for. I m...,Related,[What are the potential long-term effects of f...,"GERD, or Gastroesophageal Reflux Disease, is a...","[[“Original Answer”, “Complete Answer”], [], [..."
2,3002,eli5 Why didn’t the dwarves fight in the War o...,"They did, the movies skip a lot of side detail...",Thanks. I’m listening to the audiobooks and to...,Related,[What are some examples of the side conflicts ...,The dwarves did not actively participate in th...,"[[“Original Answer”], [“Complete Answer”], []]"
3,3003,ELI5: How does our voice turn into code / get ...,In a nutshell: The sound waves from your voic...,"Hey, I totally respect and appreciate this. Bu...",Related,[What is the role of the human ear in interpre...,"When you speak into a phone, your voice genera...","[[], [""Original Answer"", ""Complete Answer""], [..."
4,3004,eli5 how is the basic of minecraft. game....li...,"The endgame is to get to ""the end"" where there...",Wasn't the End dimension only added to minecra...,Slightly Related,"[Can you explain what ""sandbox game"" means in ...",Minecraft is an open-world sandbox game where ...,"[[""Original Answer"", ""Complete Answer""], [""Com..."
...,...,...,...,...,...,...,...,...
496,3496,ELI5: Why does storage in a hard-drive run out...,"Well, technically, they are tangible. You pro...",So when you save something it basically sends ...,Related,[What are the consequences of data overwriting...,A hard drive runs out of space because it has ...,"[[[], [""Complete Answer""]], [[], [""Complete An..."
497,3497,ELI5: Why does coin scratcher work on vending ...,>\tBut why does it help? **It doesn’t.** It i...,"Wow, that’s really interesting. I was always l...",Related,[How does the plate help in preventing vandali...,Coin scratchers work on vending machines due t...,"[[""Original Answer""], [], [""Original Answer""],..."
498,3498,"ELI5: If space-time is curved by mass, does th...",Why would it require some continuous transfer ...,>Why would it require some continuous transfer...,Related,[What is the significance of the curvature of ...,"No, the curvature of space-time caused by mass...","[[""Original Answer"", ""Complete Answer""], [""Com..."
499,3499,Eli5: What actually happens when we inadverten...,"My body doesn’t jerk, but about 5 minutes aft...",Do you suffer from anxiety or stress? This hap...,Slightly Related,[What is the role of the brainstem in initiati...,"The sensation of suddenly being ""shocked"" awak...","[[ ""Complete Answer""], [""Complete Answer""], []..."


In [ ]:
from collections import Counter
ERROR_MSG = "LLM failed to generate a response"


one = full_df_with_answerability.head(2).loc[:, 'generated_follow_up_answerability']
two = org_df_with_answerability.head(2).loc[:, 'generated_follow_up_answerability']
three = gpt_df_with_answerability.head(2).loc[:, 'generated_follow_up_answerability']

full_df_with_answerability

def analyze_informativeness(df):
    counter = Counter()
    
    for data in df:
        follow_up_answerability = data.replace(" “", '"').replace("”", '"').replace("'", '"')
        print(follow_up_answerability)
        if follow_up_answerability == ERROR_MSG: continue

        nested_list = json.loads(follow_up_answerability)
        counter.update(tuple(sublist) for sublist in nested_list)

    return counter

full_result = analyze_informativeness(one)
org_result = analyze_informativeness(two)
gpt_result = analyze_informativeness(three)

[[], ["Complete Answer"], [], ["Complete Answer"]]
[["Complete Answer"], [], ["Original Answer"]]
[[], ["Original Answer", "Complete Answer"], [], ["Complete Answer"]]
[["Original Answer", "Complete Answer"], [], ["Complete Answer"], ["Original Answer", "Complete Answer"], ["Complete Answer"]]
[["Original Answer", "Complete Answer"], ["Original Answer", "Complete Answer"], ["Complete Answer"], []]
[[ "Complete Answer"], [], [], ["Original Answer", "Complete Answer"]]


In [ ]:
print(index_raises_error)

one = full_df_with_answerability.head(10).loc[:, 'generated_follow_up_answerability']
two = org_df_with_answerability.head(10).loc[:, 'generated_follow_up_answerability']
three = gpt_df_with_answerability.head(10).loc[:, 'generated_follow_up_answerability']

pd.concat([one, two, three], axis=1)

In [58]:
### Save results into json file:
full_df_with_answerability.to_json('informativeness_output/full_df_with_answerability.json', orient='records', indent=4)
org_df_with_answerability.to_json('informativeness_output/org_df_with_answerability.json', orient='records', indent=4)
gpt_df_with_answerability.to_json('informativeness_output/gpt_df_with_answerability.json', orient='records', indent=4)

In [ ]:
# Print sample response
print(gpt_df_with_answerability.loc[7, "answer"])
print(gpt_df_with_answerability.loc[7, "complete_answer"])
print(gpt_df_with_answerability.loc[7, "generated_follow_up"])
print(gpt_df_with_answerability.loc[7, "generated_follow_up_answerability"])

Sufficiently hot objects produce their own light, a process called “blackbody radiation”  Torches, incandescent bulbs, the sun - they’re not reflecting light from another source, but rather generating light when very hot molecules dump energy to cool off.
We are able to see the white light from a torch mid-air for several reasons. Firstly, the light interacts with particles in the air, such as dust and water droplets, which reflect and scatter the light in different directions. When these scattered light rays enter our eyes, we perceive the beam as visible. 

Additionally, white light consists of multiple wavelengths, including blue light, which scatters more efficiently in the atmosphere due to the presence of gases and small particles. This effect contributes to the beam's visibility.

Furthermore, in darker environments, the contrast between the bright light beam and the surrounding darkness makes the beam stand out more clearly to our eyes. Our eyes can detect the direct path of th